In [1]:
%matplotlib inline

SUDOKU_PATH = '/home/ajhnam/sudoku'

import numpy as np
from scipy.special import factorial
import re
import itertools
from enum import Enum
import random
import pickle
import torch
import random
import matplotlib
import matplotlib.pyplot as plt

import sys
sys.path.append(SUDOKU_PATH + '/src/sudoku')

from board import Board
from solutions import Solutions
import utils
from grid_string import GridString, read_solutions_file, write_solutions_file
from dataset import Dataset, Datasets
from utils import print

In [2]:
# set random seed to 0
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
filename = SUDOKU_PATH + '/data/shuffled_puzzles.txt'
with open(filename) as f:
    lines = f.read().splitlines()
all_puzzles = {}
for line in lines:
    puzzle, solution = line.split(',')
    all_puzzles[GridString(puzzle)] = GridString(solution)
    
puzzles_by_hints = {i: {} for i in range(4, 17)}
num_cells = 16
for p in all_puzzles:
    hints = num_cells - p.grid.count('.')
    puzzles_by_hints[hints][p] = all_puzzles[p]
    
for k in sorted(puzzles_by_hints):
    print("Number of puzzles with {} hints: {}".format(k, len(puzzles_by_hints[k])))

(2018-12-23_19:25:50) Number of puzzles with 4 hints: 1536
(2018-12-23_19:25:50) Number of puzzles with 5 hints: 8568
(2018-12-23_19:25:50) Number of puzzles with 6 hints: 21192
(2018-12-23_19:25:50) Number of puzzles with 7 hints: 38016
(2018-12-23_19:25:50) Number of puzzles with 8 hints: 57216
(2018-12-23_19:25:50) Number of puzzles with 9 hints: 79416
(2018-12-23_19:25:50) Number of puzzles with 10 hints: 99576
(2018-12-23_19:25:50) Number of puzzles with 11 hints: 114096
(2018-12-23_19:25:50) Number of puzzles with 12 hints: 116184
(2018-12-23_19:25:50) Number of puzzles with 13 hints: 89784
(2018-12-23_19:25:50) Number of puzzles with 14 hints: 33384
(2018-12-23_19:25:50) Number of puzzles with 15 hints: 4608
(2018-12-23_19:25:50) Number of puzzles with 16 hints: 288


Create a training set of 250 puzzles for each number of hints from 4 to 15 for a total of 12\*250 = 3000.

Create a validation set of 50 puzzles for each number of hints from 4 to 15 for a total of 12\*50 = 600.

Create a test set of 100 puzzles for each number of hints from 4 to 15 for a total of 12\*100 = 1200.

In [4]:
# datasets = Datasets({i: Dataset(puzzles_by_hints[i]) for i in range(4, 16)})
# datasets.save('./data/datasets.pkl')
datasets = datasets.load('./data/datasets.pkl')